# Euclidean Geometry by high-peformance SMT solvers?

### Siddhartha Gadgil and Anand Rao Tadipatri

In [102]:
# %pip install z3-solver
from z3 import *
set_param(proof=True)

---
## Warmup: A simple problem

As a warmup and sanity check, consider
the problem of showing that for an arbitrary point $P = (x, y)$, the three points
$P = (x, y)$, $O = (0, 0)$ and $−P = (−x, −y)$ are collinear.

In [103]:
P = (x, y) = Reals('x y')  #the coordinates of the point P
O = (0, 0)  #the coordinates of the origin
Q = (-x, -y)  #the reflection of the point P about the origin

#### Equations for collinearity

The condition for three points $(x_1, y_1), (x_2, y_2) \text{ and } (x_3, y_3)$ being collinear is 

$$
\frac{y_2 - y_1}{x_2 - x_1} = \frac{y_3 - y_1}{x_3 - x_1}
$$

Intuitively, this says that the slope of the line joining the points $(x_1, y_1)$ and $(x_2, y_2)$ is equal to the slope of the line joining $(x_1, y_1)$ and $(x_3, y_3)$.

The above expression is equivalent to

$$(y_2 - y_1) \cdot (x_3 - x_1) = (y_3 - y_1) \cdot (x_2 - x_1)$$

In [104]:
def are_collinear(p, q, r):
    """
    Checks if three points - `p`, `q`, `r` - are collinear.
    
    Here, `p[0]` and `p[1]` denote the *x* and *y* coordinates of `p` respectively.
    """
    return ( (q[1]-p[1])*(r[0]-p[0]) == (r[1]-p[1])*(q[0]-p[0]) )

In [105]:
prove((are_collinear(P, O, Q)))

proved


This shows that the claim that the points $P$, $O$ and $Q$ are collinear is true.

Internally, the `prove` function works roughly in the following way:
- The given claim (that the points `P`, `O` and `Q` are collinear is first negated.
- The solver then checks whether the given system of equations is satisfiable, i.e., whether there are real numbers `x` and `y` for which `Not(are_collinear((x, y), (0, 0), (-x, -y))` holds.
- If no such solutions are found, this shows by contradiction that the initial claim was correct, namely that for any point `(x, y)`, the points `(x, y), (0, 0), (-x, -y)` are collinear.

In [109]:
collinearity_solverver = Solver()
collinearity_solver.add(Not(are_collinear(P, O, Q)))  #the negation of the statement
collinearity_solver.check()  #`unsat` indicates that the given equation is not satisfiable

unsat

This is the statement given to the solver

In [107]:
collinearity_solver  #the statement of the claim

[Not((0 - y)*(-x - x) == (-y - y)*(0 - x))]

This is the code in SMT2 format

In [108]:
print(collinearity_solver.sexpr())  #this is how the code is represented in SMT2 format

(declare-fun x () Real)
(declare-fun y () Real)
(assert (let ((a!1 (= (* (- 0.0 y) (- (- x) x)) (* (- (- y) y) (- 0.0 x)))))
  (not a!1)))



One can also use the Z3 solver to produce a proof of the result using the solver, which can then be verified independently to ensure that it is correct.

In [110]:
collinearity_solver.proof()

mp(asserted(Not((0 - y)*(-x - x) == (-y - y)*(0 - x))),
   trans(monotonicity(trans(monotonicity(trans(trans(monotonicity(trans(rewrite(0 -
                                        y ==
                                        0 + -1*y),
                                        rewrite(0 + -1*y ==
                                        -1*y),
                                        0 - y == -1*y),
                                        trans(trans(monotonicity(rewrite(-x ==
                                        -1*x),
                                        -x - x == -1*x - x),
                                        rewrite(-1*x - x ==
                                        -1*x + -1*x),
                                        -x - x ==
                                        -1*x + -1*x),
                                        rewrite(-1*x + -1*x ==
                                        -2*x),
                                        -x - x == -2*x),
                                        (0 - y)*(-x - x) ==
                                        -1*y*-2*x),
                                        rewrite(-1*y*-2*x ==
                                        2*y*x),
                                        (0 - y)*(-x - x) ==
                                        2*y*x),
                                        rewrite(2*y*x ==
                                        2*x*y),
                                        (0 - y)*(-x - x) ==
                                        2*x*y),
                                        trans(trans(monotonicity(trans(trans(monotonicity(rewrite(-y ==
                                        -1*y),
                                        -y - y == -1*y - y),
                                        rewrite(-1*y - y ==
                                        -1*y + -1*y),
                                        -y - y ==
                                        -1*y + -1*y),
                                        rewrite(-1*y + -1*y ==
                                        -2*y),
                                        -y - y == -2*y),
                                        trans(rewrite(0 - x ==
                                        0 + -1*x),
                                        rewrite(0 + -1*x ==
                                        -1*x),
                                        0 - x == -1*x),
                                        (-y - y)*(0 - x) ==
                                        -2*y*-1*x),
                                        rewrite(-2*y*-1*x ==
                                        2*y*x),
                                        (-y - y)*(0 - x) ==
                                        2*y*x),
                                        rewrite(2*y*x ==
                                        2*x*y),
                                        (-y - y)*(0 - x) ==
                                        2*x*y),
                                        ((0 - y)*(-x - x) ==
                                        (-y - y)*(0 - x)) ==
                                        (2*x*y == 2*x*y)),
                            rewrite((2*x*y == 2*x*y) == True),
                            ((0 - y)*(-x - x) ==
                             (-y - y)*(0 - x)) ==
                            True),
                      Not((0 - y)*(-x - x) ==
                          (-y - y)*(0 - x)) ==
                      Not(True)),
         rewrite(Not(True) == False),
         Not((0 - y)*(-x - x) == (-y - y)*(0 - x)) == False),
   False)

---
## Menelaus' theorem

In [75]:
#the description of the theorem will be later copied from the article

In [76]:
#A, B, C are the vertices of the triangle
A = (x_a, y_a) = Reals('x_a y_a')
B = (x_b, y_b) = Reals('x_b y_b')
C = (x_c, y_c) = Reals('x_c y_c')

"""
Each of the edges of the triangle can be parameterised by a single variable,
which is equal to the first vertex at 0 and equal to the second vertex at 1.
"""

r, s, t = Reals('r s t')

def cut(l, P, Q):
    """
    With the line PQ parameterised as described above,
    this function returns the point R obtained when the parameter is equal to `l`
    """
    return (P[0] + l*(Q[0] - P[0]), P[1] + l*(Q[1] - P[1]))

# D, E, F are points on the edges AB, BC, CA respectively
D, E, F = cut(r, A, B), cut(s, B, C), cut(t, C, A)

def d(p, q):
    """
    Returns the square of the Euclidean distance between two points.
    """
    return (p[0] - q[0])**2 + (p[1] - q[1])**2

def in_bounds(l):
    """
    Checks whether the parameter is within the range (0, 1),
    i.e, whether the point corresponding to the parameter value `l` is 
    contained within the corresponding edge or on an extension of it.
    """
    return And(0 < l, l < 1)

odd_not_in_bounds = Xor(Xor(Not(in_bounds(r)), Not(in_bounds(s))), Not(in_bounds(t)))

dist_eq = d(A, D) * d(B, E) * d(C, F) == d(D, B) * d(E, C) * d(F, A)

These are the forward and reverse implication parts of the theorem statement.

In [77]:
menelaus_theorem_fwd = Implies(dist_eq, are_collinear(D, E, F))
menelaus_theorem_rev = Implies(are_collinear(D, E, F), dist_eq)

menelaus_theorem = And(menelaus_theorem_fwd, menelaus_theorem_rev)

In [80]:
set_param(proof = False)

menelaus_solver = Solver()

menelaus_solver.add(Not(are_collinear(A, B, C)))
menelaus_solver.add(odd_not_in_bounds == True)
menelaus_solver.add(Not(menelaus_theorem))

menelaus_solver.check()

unsat

As before, since the negation of the theorem is unsatisfiable, the theorem must be true.

The first line of the above cell - `set_param(proof = False)` - asks the solver to check satisfiability without trying to produce a proof. If one asks for a proof by changing the line to - `set_param(proof = True)` - the solver times out after a few seconds and returns `unknown`.

Thus, the Z3 solver can be used to *solve*, but not *prove* Menelaus' theorem.

---
## Pappus' theorem

In [82]:
#the description of the theorem will be later copied from the article

In [84]:
for s in ('a', 'b', 'c', 'A', 'B', 'C', 'P', 'Q', 'R'):
    #initialise the variables
    exec("x_{n} = Real('x_{n}')".format(n = s))
    exec("y_{n} = Real('y_{n}')".format(n = s))

    #create the point
    exec("{v} = (x_{n}, y_{n})".format(v = s, n = s))

u, v, U, V = Reals('u v U V')  #the scaling parameters

a, b, c = (1, 0), (1+u, 0), (1+u+v, 0)  #the points on the first line
A, B, C = A, (x_A*(1+U), y_A*(1+U)), (x_A*(1+U+V), y_A*(1+U+V))  #the points on the second line

def all_distinct(pts):
    """
    Checks whether all points in the list `pts` are distinct.
    """
    return And([Or(Not(p[0] == q[0]), Not(p[1] == q[1])) for (i, p) in enumerate(pts) for (j, q) in enumerate(pts) if j < i])

def parallel(p_1, p_2, q_1, q_2):
    """
    Checks whether the line passing through p_1 and p_2 is parallel to the line passing through q_1 and q_2.
    This is done by checking whether the slopes of the two lines are identical.
    """
    return (p_2[1] - p_1[1])*(q_2[0] - q_1[0]) == (p_2[0] - p_1[0])*(q_2[1] - q_1[1])

pappus_theorem = Implies(And([are_collinear(p, q, r) for (p, q, r) in (
    (a, b, c), (A, B, C),
    (A, b, P), (B, a, P),
    (B, c, Q), (C, b, Q),
    (C, a, R), (A, c, R)
)]), are_collinear(P, Q, R))

In [93]:
set_param(proof = False)

pappus_solver = Solver()

pappus_solver.add(u > 0)
pappus_solver.add(v > 0)
pappus_solver.add(U > 0)
pappus_solver.add(V > 0)
pappus_solver.add(all_distinct([a, b, c]))
pappus_solver.add(all_distinct([A, B, C]))
pappus_solver.add(Not(parallel(a, b, A, B)))

pappus_solver.add(Not(pappus_theorem))

pappus_solver.check()

unsat

As with Menelaus' theorem, Pappus' theorem can be solved by the Z3 solver, but not proved.

---